# reading data from dataset

In [53]:
#  read files from cnn and dailymail and add them to a dictionary where key is file name and values are list of 
#  sentences in that file.
from os import listdir,getcwd
from os.path import isfile, join
def readCorpus(datasetDirectoryName):
    docSummaryDict = {}
    docSentencesDict = {}
    summaryline = False
    # datasetDirectoryName = ['cnn', 'dailymail']
    for dirName in datasetDirectoryName:
        filepath = join(getcwd(), dirName,'stories')
        allFiles = [f for f in listdir(filepath) if not (f.startswith('.')) and isfile(join(filepath, f))]
        for file in allFiles:
            filename = join(filepath, file)
            if 'cnn' in filename:
                fileKey = 'cnn_' + file
            else:
                fileKey = 'dailymail_' + file
            for line in open(filename,'r'): 
                if line != '\n':
                    if '@highlight' in line:
                        summaryline = True
                        continue
                    if not summaryline:
                        if fileKey in docSentencesDict:
                            docSentencesDict[fileKey].append(line.strip())
                        else:
                            docSentencesDict[fileKey] = [line.strip()]
                    else:
                        summaryline = False
                        if fileKey in docSummaryDict:
                            docSummaryDict[fileKey].append(line.strip())
                        else:
                            docSummaryDict[fileKey] = [line.strip()]
    return docSentencesDict, docSummaryDict 

# Term Doc Frequency dictionary

In [54]:
# we are fetching words from wiki dumps
from os import getcwd
from os.path import join
def generateIdfDict():
    idfFile = 'finalIdfscore.csv'
    file = join(getcwd(), idfFile)
    idfDict = {}
    f = open(file,'r')
    for line in f:
        lineArr = line.split(',')
        idfDict[lineArr[0]] = float(lineArr[1])
    f.close()
    return idfDict

In [132]:
#  funciton to get term-document frequency
#  docSentencesDict should be stemmed words dictionary
#  returns you count of words appear in how many documents for idf
# from collections import Counter
# from nltk import word_tokenize
# def getTermDocfrequency(docSentencesDict):
#     termDocFreqDict = Counter()
# #     each k is a unique file and v is list of sentences
#     for k in docSentencesDict:
#         v = docSentencesDict[k]
#         fileSet = None
#         for line in v:
#             lineSet = {word for word in word_tokenize(line)}
# #             print(lineSet)
#             if fileSet is None:
#                 fileSet = lineSet
#             else:
#                 fileSet = fileSet | lineSet
#         for word in fileSet:
#             termDocFreqDict[word] += 1
#     return termDocFreqDict        

In [111]:
#  funciton to get term-document frequency
#  docSentencesDict should be stemmed words dictionary
#  returns you count of words appear in how many documents for idf
# from collections import Counter
# from nltk import word_tokenize
# def getTermDocfrequency(textArr):
#     termDocFreqDict = Counter()
# #     each sentence is considered as a unique document
#     for line in textArr:
# #     naming it as file set because each line is considered as an individual file
#         fileSet = {word for word in word_tokenize(line)}
#         for word in fileSet:
#             termDocFreqDict[word] += 1
#     return termDocFreqDict

# remove stop words

In [55]:
# function to read stop words from a text file
def readStopWords(filename):
    text = []
    if(filename.endswith('txt')):
        file = open(filename, 'r')
        for line in file:
            text.append(line.strip())
        return text
    else:
        return None;


from nltk import word_tokenize
def removeStopWords(docSentencesDict):
    returnDocSentencesDict = {}
#     returnText = ''
 #  list of stopWords ( removed following words from list -> eight, eleven, fifteen, first, five, forty, four, nine, 
#   one, six, sixty, twelve, twenty, two, ten, )
# http://xpo6.com/download-stop-word-list/
    stopWordsFile = 'stop-word-list.txt'
    stopWords = readStopWords(stopWordsFile)
    if stopWords is None:
        raise Exception('Couldn\'t parse the given file. Stop Words list is empty. Please provide a text file to parse.')
#     print(stopWords)
    for file in docSentencesDict:
        modifiedTextArr = []
        textArr = docSentencesDict[file]
        for text in textArr:
            modifiedText = ''
            words = word_tokenize(text)
            for word in words:
                if word not in stopWords:
                    modifiedText += word + ' '
            modifiedTextArr.append(modifiedText)
        returnDocSentencesDict[file] = modifiedTextArr
    return returnDocSentencesDict     

In [2]:
# print(removeStopWords('This is four times better than previous solution'))

This four times better previous solution 


# Stemming

In [56]:
# nltk.download()
# creating stemmed text and stemmedTextDict(for term frequency)
from nltk.stem import PorterStemmer
from nltk import word_tokenize
def getStemmedText(docSentencesDict):
    returnedDocSentencesDict = {}
    stemmedWordFrequencyDict = {}
    ps = PorterStemmer()
    for file in docSentencesDict:
        modifiedTextArr = []
        textArr = docSentencesDict[file]
        for text in textArr:
            stemmedText = ''
#         text = docSentencesDict[file]
        
            words = word_tokenize(text)
            for word in words:
                stemmedWord = ps.stem(word)
                stemmedText += stemmedWord + ' '
                if(stemmedWordFrequencyDict.get(stemmedWord, None) == None):
                    stemmedWordFrequencyDict[stemmedWord] =1
                else:
                    stemmedWordFrequencyDict[stemmedWord] += 1 
            modifiedTextArr.append(stemmedText)
        returnedDocSentencesDict[file] = modifiedTextArr
    return returnedDocSentencesDict, stemmedWordFrequencyDict

In [5]:
# print(getStemmedText('python pythonly pythoning pythoned jatin sankar. jatin Sankar python'))

# Data set downloaded using following link
# https://cs.nyu.edu/~kcho/DMQA/

# Computing Lex Rank Scores

In [57]:
from nltk import word_tokenize
from collections import Counter
from math import pow, log, sqrt
from nltk import word_tokenize
def idfModifiedCosine(sentence1,sentence2, idfDict):
#     print('sentence1 is ' + sentence1)
#     print('sentence2 is ' + sentence2)
    sent1TFDict = Counter()
    sent2TFDict = Counter()
#     idfDict = {}
    wordsInbothSentences = None
    sentence1Arr = word_tokenize(sentence1)
    sentence2Arr = word_tokenize(sentence2)
    for word in sentence1Arr:
        sent1TFDict[word] += 1
        if wordsInbothSentences is None:
            wordsInbothSentences = {word}
        else:
            wordsInbothSentences = wordsInbothSentences | {word}
#         if word not in idfDict:
# #             print('calculate idf for word : ' +  word + ' ' +  str(termDocfreqDict[word]))
#             idfDict[word] = log(totalNumberOfDocs / termDocfreqDict[word])
    for word in sentence2Arr:
        sent2TFDict[word] += 1
        if wordsInbothSentences is None:
            wordsInbothSentences = {word}
        else:
            wordsInbothSentences = wordsInbothSentences | {word}
#         if word not in idfDict:
# #             print('calculate idf for word : ' +  word + ' ' +  str(termDocfreqDict[word]))
#             idfDict[word] = log(totalNumberOfDocs / termDocfreqDict[word])
    num  = 0
    denSent1 = 0
    denSent2 = 0
    for word in wordsInbothSentences:
        num += sent1TFDict.get(word,0) * sent2TFDict.get(word,0) * pow(idfDict.get(word,1),2)
        denSent1 += pow(sent1TFDict.get(word,0) * idfDict.get(word,1),2)
        denSent2 += pow(sent2TFDict.get(word,0) * idfDict.get(word,1),2)              
    return num / (sqrt(denSent1) * sqrt(denSent2))

In [58]:
#  implementing matrix Product
import numpy as np
def matrixProduct(matA, matB):
    return np.matmul(matA, matB)

In [59]:
import numpy as np
def matrixTranspose(mat):
    return np.transpose(mat)

In [60]:
import numpy as np
def matrixDifference(mat1,mat2):
    return np.subtract(mat1,mat2)

In [61]:
from math import pow
# http://www.personal.soton.ac.uk/jav/soton/HELM/workbooks/workbook_30/30_4_matrx_norms.pdf
def calculateEucledeanNorm(mat):
    val = 0
    for row in mat:
        for col in range(len(row)):
            val += pow(row[col],2)
    return pow(val,1/2)

In [62]:
#  implementing power method
def powerMethod(cosineMatrix, N, tolerance):
    cosineMatrixTranspose = matrixTranspose(cosineMatrix)
    initializeP = [[(1/N) for x in range(N)] for y in range(N)]
    t = 0
    currentP = None
    while True:
        if currentP is None:
            oldP = initializeP
        else:
            oldP = currentP
        t += 1
        currentP = matrixProduct(cosineMatrixTranspose,oldP)
        differenceMatrix = matrixDifference(currentP, oldP)
        difference = calculateEucledeanNorm(differenceMatrix)      
        if difference < tolerance:
            return currentP      

In [7]:
# # split Text into sentences
# def SplitTextIntoSentences(text):
#     updatedText = ''
#     for i in range(1,len(text)):
#         if text[i-1] == '.' and text[i] == ' ':
#             updatedText += '\n'
#         else:
#             updatedText += text[i]
#     sentences = updatedText.split('\n')
#     return sentences

In [63]:
#  Lex Rank
#  need to check from where do we get tolerance, threshold
#  idf-modified-cosine
#  We might get divide by zero error, need to think how to avoid that.

def lexRank(sentences,threshold, tolerance, idfDict):
    n = len(sentences)
    cosineMatrix = [[0 for x in range(n)] for y in range(n)]
    degree = [1 for x in range(n)]
    for i in range(0,n):
        for j in range(0,n):
            cosineMatrix[i][j] = idfModifiedCosine(sentences[i],sentences[j], idfDict)
            if cosineMatrix[i][j] > threshold:
                cosineMatrix[i][j] = 1
                degree[i] += 1
            else:
                cosineMatrix[i][j] = 0
    for i in range(0,n):
        for j in range(0,n):
            cosineMatrix[i][j] = cosineMatrix[i][j]/degree[i]
    L = powerMethod(cosineMatrix, n, tolerance)
    return L, degree

# write results to disk

In [64]:
#  storing summarisation to disk
from os import listdir,getcwd
from os.path import isfile, join
def writeResults(filename, summarySentences, subfolder):
    dirName = 'summarisation'
    file = join(getcwd(), dirName, subfolder, filename)
    f = open(file,'w')
    for line in summarySentences:
        f.write(line + '\n')
    f.close()

# Commands to run Lex Rank

In [ ]:
# docSentencesDict = readCorpus(['Test_cnn', 'Test_dailymail'])
# #  preprocessing
# removedStopWordsDocSentencesDict = removeStopWords(docSentencesDict)
# # stemmedWordFrequencyDict is not need in our implementation
# stemmedDocSentencesDict, stemmedWordFrequencyDict = getStemmedText(removedStopWordsDocSentencesDict)
# idfDict = getTermDocfrequency(stemmedDocSentencesDict)
# totalDocs = len(docSentencesDict)
# # testFile will be array of stemmed sentences in a test file
# # testFile = 
#  threshold is in range of [0.1 - 0.3]
#  very high thresholds may lose almost all of the information in a similarity matrix
# threshold = 0.1
#  tolerance is also consider as damping factor, range [0.1 - 0.2], research paper has taken it as 0.85
# tolerance = 0.1
# L = lexRank(testFile,threshold, tolerance, totalDocs, termDocFrequencyDict)

In [ ]:
# Questions are
# 1) threshold value
# 2) how will I convert L returned by lex rank to readable summary

# Testing the code on first 5 files of cnn and dailymail

In [95]:
testDocSentencesDict, testDocSummaryDict = readCorpus(['Test_cnn'])
# testDocSentencesDict

In [96]:
# testDocSummaryDict

In [97]:
testRemovedStopWordsDocSentencesDict = removeStopWords(testDocSentencesDict)
# testRemovedStopWordsDocSentencesDict

In [98]:
testStemmedDocSentencesDict, testStemmedWordFrequencyDict = getStemmedText(testRemovedStopWordsDocSentencesDict)
# testStemmedDocSentencesDict

In [69]:
#  our whole corpus is just 1 document that we need to sumamrise
#  total number of docs will be number of sentences

# allSentences = []
# for file in list(testStemmedDocSentencesDict.keys()):
#     allSentences.extend(testStemmedDocSentencesDict[file])
# testTotalDocs = len(allSentences)
# testTermDocFrequencyDict = getTermDocfrequency(allSentences)

# testTermDocFrequencyDict = getTermDocfrequency(testStemmedDocSentencesDict)
# for i in testTermDocFrequencyDict:
#     if testTermDocFrequencyDict[i] == 0:
#         print(i) 

idfDict = generateIdfDict()
idfDict

{'the': 0.09651045205121467,
 'in': 0.1377742631939715,
 'a': 0.1476122422408334,
 'of': 0.1839980234436306,
 'is': 0.24948374340318064,
 'and': 0.26525009831276236,
 'to': 0.367056286323574,
 'was': 0.432354514294052,
 'by': 0.5975081239009793,
 'as': 0.5987180454036926,
 'for': 0.5999898282797145,
 'on': 0.6075066476506442,
 'it': 0.6105390998233156,
 'with': 0.6633704325148753,
 'at': 0.6937701835115793,
 'from': 0.6958574529177237,
 'an': 0.7531871716715788,
 "'s": 0.8540803288399521,
 'that': 0.9678089998449518,
 'also': 1.0001204191043205,
 'which': 1.0335289581562295,
 'has': 1.145411957273002,
 'he': 1.1549302868938929,
 'be': 1.1695056208822234,
 'first': 1.170284313266806,
 'this': 1.1918052438659628,
 'his': 1.1943532597811413,
 'one': 1.2172875609797342,
 'are': 1.2593737171741626,
 'were': 1.271638416721268,
 'after': 1.2809021679954333,
 'who': 1.2815751357212855,
 'includ': 1.312172823024545,
 'year': 1.3273948173185584,
 'two': 1.360118964048718,
 'had': 1.3720759763669

In [114]:
# testTotalDocs

56

In [86]:
# testTotalDocs = len(testDocSentencesDict)
# testTotalDocs
# testDocSentencesDict

In [99]:
OriginaltestFile = testDocSentencesDict['cnn_NewsFile4.txt']
testFile = testStemmedDocSentencesDict['cnn_NewsFile4.txt']
# testFile

In [100]:
#  threshold is in range of [0.1 - 0.3]
#  very high thresholds may lose almost all of the information in a similarity matrix
threshold = 0.1
#  tolerance is also consider as damping factor, range [0.1 - 0.2], research paper has taken it as 0.85
tolerance = 0.1
# L, degree = lexRank(allSentences,threshold, tolerance, testTotalDocs, testTermDocFrequencyDict)
L, degree = lexRank(testFile, threshold, tolerance, idfDict)
print(degree)
print(L)

[9, 3, 14, 10, 4, 12, 3, 5, 17, 20, 11, 13, 4, 9, 13, 15, 14, 11, 16, 9, 13, 18, 4, 11, 9, 12, 13, 5, 5, 11, 4, 3, 5, 9, 4, 14, 16, 15, 11, 7, 22, 6, 5, 3, 2, 14, 17]
[[ 0.01222545  0.01222545  0.01222545 ...,  0.01222545  0.01222545
   0.01222545]
 [ 0.00588997  0.00588997  0.00588997 ...,  0.00588997  0.00588997
   0.00588997]
 [ 0.01921814  0.01921814  0.01921814 ...,  0.01921814  0.01921814
   0.01921814]
 ..., 
 [ 0.00265957  0.00265957  0.00265957 ...,  0.00265957  0.00265957
   0.00265957]
 [ 0.01902912  0.01902912  0.01902912 ...,  0.01902912  0.01902912
   0.01902912]
 [ 0.02358857  0.02358857  0.02358857 ...,  0.02358857  0.02358857
   0.02358857]]


In [101]:
i=0
for row in L:
#     for col in row:
#         print(str(col)) + ('  ')
    print(str(row[0]) + '    ' + str(i))
    i += 1
#     print('\n')

0.0122254545208    0
0.00588997241923    1
0.0192181395472    2
0.013260949893    3
0.0043064935737    4
0.0162429678863    5
0.00399459217418    6
0.00935158206178    7
0.0238307110616    8
0.0287400836516    9
0.0156674523658    10
0.0208603219524    11
0.00630621017599    12
0.0123463197682    13
0.0180743270361    14
0.0222594884683    15
0.0199691112175    16
0.0159403336245    17
0.0227521363208    18
0.0122132802051    19
0.0184312532894    20
0.0254672888852    21
0.00893534430503    22
0.0153292593493    23
0.0122132802051    24
0.0170177126381    25
0.0177923036553    26
0.00736034812363    27
0.00604944699737    28
0.0163866596495    29
0.00551714380511    30
0.00349868894313    31
0.00635770146535    32
0.012381117747    33
0.0044876462641    34
0.0193544203494    35
0.0234392538965    36
0.022952365449    37
0.0160367904621    38
0.00893006001078    39
0.0313690260476    40
0.00893717300483    41
0.0112715423378    42
0.00570100624356    43
0.00265957446809    44
0.0190291

In [102]:
maxVal = 0
for row in L:
    if row[0] > maxVal:
        maxVal = row[0]    

In [32]:
# cutOff = 20
# systemGenerated = []
# cutOffVal = ((100 - cutOff)/100) * maxVal
# print(cutOffVal)
# i = 0
# for row in L:
#     if row[0] > cutOffVal:
#         systemGenerated.append(testFile[i])
#         print(testFile[i] + '   '+ str(i))
#     i += 1

In [103]:
cutOff = 20
systemGenerated = []
cutOffVal = ((100 - cutOff)/100) * maxVal
print(cutOffVal)
i = 0
for row in L:
    if row[0] > cutOffVal:
        systemGenerated.append(OriginaltestFile[i])
        print(OriginaltestFile[i] + '   '+ str(i))
    i += 1

0.0250952208381
"I figured the best way to test the waters was with a pseudonym," said the Burlington, Vermont, resident who asked that her name not be published to preserve the anonymity of her Twitter persona. "I wouldn't offend friends and colleagues if I didn't follow them, and I could make a polite exit from Twitter when I was done poking around."   9
The Orange Cone was originally a MySpace account but came to Twitter after its creator discovered the witty tweets of Bronx Zoo's Cobra, last year's famously escaped snake, and motorsports satirist @nascarcasm. Today the Orange Cone keeps a following of more than 20,000 users up to date on NASCAR happenings.   21
Other Twitter users say that embodying nonhuman entities gives them the chance to create alter egos. "The Cone gets to say and do the things I think we all wish we could do," its creator said.   40


In [105]:
testDocSummaryDict['cnn_NewsFile4.txt']

['Some popular Twitter feeds personify animals or inanimate objects',
 'The London Olympics inspire nonhuman accounts such as @OlympicSeat, @LeyvasTowel',
 'Other accounts represent cats, horses, traffic cones and even the universe',
 'The average Twitter user has 126 followers, but some novelty accounts have 1 million+']

In [65]:
len(testDocSentencesDict['dailymail_0a0a733db965c3fdf9bc2895104a1ef884a3d593.story'])
# sum(testDocSentencesDict.values())

56

In [106]:
# systemGenerated.txt,humanGenerated.txt
writeResults('cnn_NewsFile4.txt', systemGenerated, 'systemGenerated')

In [108]:
writeResults('cnn_NewsFile4.txt', testDocSummaryDict['cnn_NewsFile4.txt'], 'humanGenerated')